Creating MySQL Connection

In [27]:
import mysql.connector

host = "localhost"
user = "root"
password = "K@run@28"

conn = None

try:
    conn = mysql.connector.connect(
        host=host,
        user=user,
        password=password
    )
    cursor = conn.cursor()
    
    cursor.execute("CREATE DATABASE IF NOT EXISTS vaccination_data_analysis")
    conn.commit()
    print("Database created successfully.")

except mysql.connector.Error as e:
    print(f"Error creating database: {e}")
finally:
    if conn:
        conn.close()


Database created successfully.


Create Tables

In [28]:
host = "localhost"
user = "root"
password = "K@run@28"
database = "vaccination_data_analysis"

try:
    conn = mysql.connector.connect(
        host=host,
        user=user,
        password=password,
        database=database
    )
    cursor = conn.cursor()

    # Table 1: Coverage Data
    create_coverage_table = """
    CREATE TABLE Coverage (
        `GROUP` VARCHAR(255),
        CODE VARCHAR(100),
        NAME VARCHAR(255),
        YEAR INT,
        ANTIGEN VARCHAR(255),
        ANTIGEN_DESCRIPTION TEXT,
        COVERAGE_CATEGORY VARCHAR(255),
        COVERAGE_CATEGORY_DESCRIPTION TEXT,
        TARGET_NUMBER BIGINT,
        DOSES INT,
        COVERAGE FLOAT
    );
    """
    cursor.execute(create_coverage_table)

    # Table 2: Incidence Rate
    create_incidence_rate_table = """
    CREATE TABLE IncidenceRate (
        `GROUP` VARCHAR(255),
        CODE VARCHAR(100),
        NAME VARCHAR(255),
        YEAR INT,
        DISEASE VARCHAR(255),
        DISEASE_DESCRIPTION TEXT,
        DENOMINATOR VARCHAR(255),
        INCIDENCE_RATE FLOAT
    );
    """
    cursor.execute(create_incidence_rate_table)

    # Table 3: Reported Cases
    create_reported_cases_table = """
    CREATE TABLE ReportedCases (
        `GROUP` VARCHAR(255),
        CODE VARCHAR(100),
        NAME VARCHAR(255),
        YEAR INT,
        DISEASE VARCHAR(255),
        DISEASE_DESCRIPTION TEXT,
        CASES INT
    );
    """
    cursor.execute(create_reported_cases_table)

    # Table 4: Vaccine Introduction
    create_vaccine_introduction_table = """
    CREATE TABLE VaccineIntroduction (
        ISO_3_CODE VARCHAR(100),
        COUNTRYNAME VARCHAR(255),
        WHO_REGION VARCHAR(255),
        YEAR INT,
        DESCRIPTION TEXT,
        INTRO VARCHAR(100)
    );
    """
    cursor.execute(create_vaccine_introduction_table)

    # Table 5: Vaccine Schedule
    create_vaccine_schedule_table = """
    CREATE TABLE VaccineSchedule (
        ISO_3_CODE VARCHAR(100),
        COUNTRYNAME VARCHAR(255),
        WHO_REGION VARCHAR(255),
        YEAR INT,
        VACCINECODE VARCHAR(255),
        VACCINE_DESCRIPTION TEXT,
        SCHEDULEROUNDS INT,
        TARGETPOP VARCHAR(255),
        TARGETPOP_DESCRIPTION TEXT,
        GEOAREA VARCHAR(100),
        AGEADMINISTERED VARCHAR(255),
        SOURCECOMMENT TEXT
    );
    """
    cursor.execute(create_vaccine_schedule_table)

    conn.commit()
    print("Tables created successfully.")

except mysql.connector.Error as e:
    print(f"Error creating tables: {e}")

finally:
    conn.close()

Tables created successfully.


In [29]:
import pandas as pd
import mysql.connector

def import_data_from_csv(file_path, table_name):
    try:
        conn = mysql.connector.connect(
            host=host,
            user=user,
            password=password,
            database=database
        )
        cursor = conn.cursor()

        df = pd.read_csv(file_path)
        for index, row in df.iterrows():
            values = tuple(row)
            placeholders = ', '.join(['%s'] * len(values))
            sql = f"INSERT INTO {table_name} VALUES ({placeholders})"
            cursor.execute(sql, values)
        conn.commit()
        print(f"Data imported successfully from {file_path} to {table_name}")

    except Exception as e:
        print(f"Error importing data from {file_path}: {e}")

    finally:
        conn.close()

import_data_from_csv("coverage_dataN", "Coverage")
import_data_from_csv("incidence_rate_dataN", "IncidenceRate")
import_data_from_csv("reported_cases_dataN", "ReportedCases")
import_data_from_csv("vaccine_introduction_dataN", "VaccineIntroduction")
import_data_from_csv("vaccine_schedule_dataN", "VaccineSchedule")

Data imported successfully from coverage_dataN to Coverage
Data imported successfully from incidence_rate_dataN to IncidenceRate
Data imported successfully from reported_cases_dataN to ReportedCases
Data imported successfully from vaccine_introduction_dataN to VaccineIntroduction
Data imported successfully from vaccine_schedule_dataN to VaccineSchedule


Creating separate tables

In [30]:
try:
    conn = mysql.connector.connect(
        host=host,
        user=user,
        password=password,
        database=database
    )
    cursor = conn.cursor()


# Creating Countries table
    create_countries_table = """
    CREATE TABLE Countries (
        `GROUP` VARCHAR(255),
        CODE VARCHAR(100),
        NAME VARCHAR(255),
        PRIMARY KEY (CODE) 
    );
    """
    cursor.execute(create_countries_table)

    # Insert unique country data from Coverage table
    insert_countries_from_coverage = """
    INSERT IGNORE INTO Countries (`GROUP`, CODE, NAME)
    SELECT DISTINCT `GROUP`, CODE, NAME 
    FROM Coverage;
    """
    cursor.execute(insert_countries_from_coverage)

    # Create Years table
    create_years_table = """
    CREATE TABLE Years (
        YEAR INT PRIMARY KEY
    );
    """
    cursor.execute(create_years_table)

    # Insert unique years from vaccineintroduction table
    insert_years_from_vaccineintroduction = """
    INSERT IGNORE INTO Years (YEAR)
    SELECT DISTINCT YEAR 
    FROM VaccineIntroduction;
    """
    cursor.execute(insert_years_from_vaccineintroduction)

        # Create Diseases table
    create_diseases_table = """
    CREATE TABLE Diseases (
        DISEASE VARCHAR(255) PRIMARY KEY,
        DISEASE_DESCRIPTION TEXT
    );
    """
    cursor.execute(create_diseases_table)

    # Insert unique diseases from IncidenceRate table
    insert_diseases_from_incidencerate = """
    INSERT IGNORE INTO Diseases (DISEASE, DISEASE_DESCRIPTION)
    SELECT DISTINCT DISEASE, DISEASE_DESCRIPTION 
    FROM IncidenceRate;
    """
    cursor.execute(insert_diseases_from_incidencerate)

        # Create WHO_Region table
    create_who_region_table = """
    CREATE TABLE WHO_Region (
        ISO_3_CODE VARCHAR(100),
        COUNTRYNAME VARCHAR(255),
        WHO_REGION VARCHAR(255),
        PRIMARY KEY (ISO_3_CODE, COUNTRYNAME) 
    );
    """
    cursor.execute(create_who_region_table)

    # Insert unique WHO_Region data from VaccineSchedule table
    insert_who_region_from_VaccineSchedule = """
    INSERT IGNORE INTO WHO_Region (ISO_3_CODE, COUNTRYNAME, WHO_REGION)
    SELECT DISTINCT ISO_3_CODE, COUNTRYNAME, WHO_REGION 
    FROM VaccineSchedule;
    """
    cursor.execute(insert_who_region_from_VaccineSchedule)

    conn.commit()
    print("Tables are created and data is inserted")
except mysql.connector.Error as e:
    print(f"Database error: {e}")

finally:
    conn.close()

Tables are created and data is inserted


Adding Primary & Foregin key constraints

In [31]:
try:
    conn = mysql.connector.connect(
        host=host,
        user=user,
        password=password,
        database=database
    )
    cursor = conn.cursor()
    
    # Add primary and foreign keys to Coverage table
    alter_coverage_table = """
    ALTER TABLE Coverage
    ADD FOREIGN KEY (CODE) REFERENCES Countries(CODE), 
    ADD FOREIGN KEY (YEAR) REFERENCES Years(YEAR);
    """
    cursor.execute(alter_coverage_table)

    # Add primary and foreign keys to IncidenceRate table
    alter_incidencerate_table = """
    ALTER TABLE IncidenceRate
    ADD FOREIGN KEY (CODE) REFERENCES Countries(CODE),
    ADD FOREIGN KEY (DISEASE) REFERENCES Diseases(DISEASE),
    ADD FOREIGN KEY (YEAR) REFERENCES Years(YEAR);
    """
    cursor.execute(alter_incidencerate_table)

    # Add primary and foreign keys to ReportedCases table
    alter_reportedcases_table = """
    ALTER TABLE ReportedCases
    ADD FOREIGN KEY (CODE) REFERENCES Countries(CODE),
    ADD FOREIGN KEY (DISEASE) REFERENCES Diseases(DISEASE),
    ADD FOREIGN KEY (YEAR) REFERENCES Years(YEAR);
    """
    cursor.execute(alter_reportedcases_table)

    # Add primary and foreign keys to VaccineIntroduction table
    alter_vaccineintroduction_table = """
    ALTER TABLE VaccineIntroduction
    ADD FOREIGN KEY (ISO_3_CODE, COUNTRYNAME) REFERENCES WHO_Region(ISO_3_CODE, COUNTRYNAME), 
    ADD FOREIGN KEY (YEAR) REFERENCES Years(YEAR); 
    """
    cursor.execute(alter_vaccineintroduction_table)

    # Add primary and foreign keys to VaccineSchedule table
    alter_vaccineschedule_table = """
    ALTER TABLE VaccineSchedule
    ADD FOREIGN KEY (ISO_3_CODE, COUNTRYNAME) REFERENCES WHO_Region(ISO_3_CODE, COUNTRYNAME), 
    ADD FOREIGN KEY (YEAR) REFERENCES Years(YEAR); 
    """
    cursor.execute(alter_vaccineschedule_table)

    conn.commit()
    print("Primary and foreign keys added successfully.")

except mysql.connector.Error as e:
    print(f"Error adding primary and foreign keys: {e}")

finally:
    conn.close()

Primary and foreign keys added successfully.


Drop duplicate columns

In [32]:
try:
    conn = mysql.connector.connect(
        host=host,
        user=user,
        password=password,
        database=database
    )
    cursor = conn.cursor()
    
    
    drop_columns_queries = [
        # Drop duplicate columns from Coverage
        "ALTER TABLE Coverage DROP COLUMN NAME, DROP COLUMN `GROUP`;",
        # Drop duplicate columns from IncidenceRate
        "ALTER TABLE IncidenceRate DROP COLUMN NAME, DROP COLUMN `GROUP`;",
        "ALTER TABLE IncidenceRate DROP COLUMN DISEASE_DESCRIPTION;",
        # Drop duplicate columns from ReportedCases
        "ALTER TABLE ReportedCases DROP COLUMN NAME, DROP COLUMN `GROUP`;",
        "ALTER TABLE ReportedCases DROP COLUMN DISEASE_DESCRIPTION;",
        # Drop duplicate columns from VaccineIntroduction
        "ALTER TABLE VaccineIntroduction DROP COLUMN WHO_REGION;",
        # Drop duplicate columns from VaccineSchedule
        "ALTER TABLE VaccineSchedule DROP COLUMN WHO_REGION;"
    ]
    for query in drop_columns_queries:
        try:
            cursor.execute(query)
            print(f"Executed: {query.strip()}")
        except mysql.connector.Error as e:
            print(f"Skipping column drop: {query.strip()} - Error: {e}")

    conn.commit()
    print("Duplicate columns dropped successfully where applicable.")

except mysql.connector.Error as e:
    print(f"Error processing the database: {e}")

finally:
    conn.close()

Executed: ALTER TABLE Coverage DROP COLUMN NAME, DROP COLUMN `GROUP`;
Executed: ALTER TABLE IncidenceRate DROP COLUMN NAME, DROP COLUMN `GROUP`;
Executed: ALTER TABLE IncidenceRate DROP COLUMN DISEASE_DESCRIPTION;
Executed: ALTER TABLE ReportedCases DROP COLUMN NAME, DROP COLUMN `GROUP`;
Executed: ALTER TABLE ReportedCases DROP COLUMN DISEASE_DESCRIPTION;
Executed: ALTER TABLE VaccineIntroduction DROP COLUMN WHO_REGION;
Executed: ALTER TABLE VaccineSchedule DROP COLUMN WHO_REGION;
Duplicate columns dropped successfully where applicable.
